# Baseline MLP with Arrhenius Kinetics Features

This baseline implements:
1. Arrhenius kinetics features (1/T, ln(t), interaction)
2. Spange descriptors for solvent featurization
3. MLP with BatchNorm, Dropout, Sigmoid output
4. HuberLoss for robustness

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set default dtype for numerical stability
torch.set_default_dtype(torch.double)

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

Using device: cuda
GPU: NVIDIA A100-SXM4-80GB


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def load_features(name='spange_descriptors'):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f'Spange descriptors shape: {SPANGE_DF.shape}')
print(f'Solvents available: {len(SPANGE_DF)}')

Spange descriptors shape: (26, 13)
Solvents available: 26


In [3]:
# Featurizer with Arrhenius kinetics
class KineticFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        # 2 numeric + 3 kinetic + 13 spange = 18 features
        self.feats_dim = self.featurizer.shape[1] + 2 + 3

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]  # Temperature in Celsius
        time_m = X_vals[:, 0:1]  # Residence time in minutes
        
        temp_k = temp_c + 273.15  # Convert to Kelvin
        inv_temp = 1000.0 / temp_k  # Inverse temperature (scaled)
        log_time = np.log(time_m + 1e-6)  # Log of time
        interaction = inv_temp * log_time  # Interaction term
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # Solvent features
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # Symmetry flip: swap A and B
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)

print(f'Feature dimension: {KineticFeaturizer().feats_dim}')

Feature dimension: 18


In [4]:
# MLP Model with BatchNorm, Dropout, Sigmoid output
class MLPInternal(nn.Module):
    def __init__(self, input_dim, hidden_dims=[128, 128, 64], output_dim=3, dropout=0.2):
        super(MLPInternal, self).__init__()
        layers = [nn.BatchNorm1d(input_dim)]
        
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())  # Output in [0, 1]
        
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# Main model class that follows the competition template
class BaselineMLP(nn.Module):
    def __init__(self, data='single', n_models=3):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticFeaturizer(mixed=(data=='full'))
        self.n_models = n_models
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        # Standard data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # Data augmentation: train on both symmetries
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()  # Robust to outliers
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            # Test Time Augmentation (TTA)
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

In [5]:
# Run cross-validation for single solvent data
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = BaselineMLP(data='single', n_models=3)
    model.train_model(train_X, train_Y, epochs=200)
    predictions = model.predict(test_X).numpy()
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    # Store actuals for MSE calculation
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)

# Calculate MSE for single solvent
actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:58<22:35, 58.95s/it]

  8%|▊         | 2/24 [01:40<17:55, 48.88s/it]

 12%|█▎        | 3/24 [02:19<15:32, 44.42s/it]

...

 92%|█████████▏| 22/24 [16:17<01:29, 44.64s/it]

 96%|█████████▌| 23/24 [16:58<00:43, 43.77s/it]

100%|██████████| 24/24 [17:38<00:00, 42.49s/it]

100%|██████████| 24/24 [17:38<00:00, 44.09s/it]


Single Solvent MSE: 0.010429


In [6]:
# Run cross-validation for full data (mixtures)
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = BaselineMLP(data='full', n_models=3)
    model.train_model(train_X, train_Y, epochs=200)
    predictions = model.predict(test_X).numpy()
    
    # Store predictions
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    
    # Store actuals for MSE calculation
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)

# Calculate MSE for full data
actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [02:29<29:58, 149.91s/it]

 15%|█▌        | 2/13 [05:03<27:50, 151.84s/it]

 23%|██▎       | 3/13 [07:39<25:39, 153.90s/it]

...

 85%|████████▍ | 11/13 [28:00<05:10, 155.08s/it]

 92%|█████████▏| 12/13 [30:49<02:39, 159.18s/it]

100%|██████████| 13/13 [33:26<00:00, 158.43s/it]

100%|██████████| 13/13 [33:26<00:00, 154.31s/it]


Full Data MSE: 0.011429


In [7]:
# Calculate overall MSE (weighted by number of samples)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')


=== FINAL RESULTS ===
Single Solvent MSE: 0.010429 (n=656)
Full Data MSE: 0.011429 (n=1227)
Overall MSE: 0.011081


In [8]:
# Create submission file
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved to /home/submission/submission.csv')
print(f'Submission shape: {submission.shape}')
print(submission.head())

Submission saved to /home/submission/submission.csv
Submission shape: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.011080  0.010218  0.897487
1       1     0     0    1  0.028685  0.025615  0.821114
2       2     0     0    2  0.067066  0.059342  0.693974
3       3     0     0    3  0.104709  0.081861  0.584283
4       4     0     0    4  0.125673  0.089247  0.525820
